In [12]:
import os
import pandas as pd
import numpy as np
import geocoder
import feather
import re
from tqdm import tqdm_notebook as tqdm

In [13]:
train = pd.read_csv('../input/train.csv')
target = train["賃料"]
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)


access = feather.read_dataframe("../code/feature_csv/access_feature.feather")
all_df = pd.concat([all_df, access], axis=1)
base = feather.read_dataframe("../code/feature_csv/base_feature1.feather")
all_df = pd.concat([all_df, base], axis=1)

org_columns = all_df.columns

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [14]:
def modify_miss(df_input):
    df_input.loc[df_input["間取り"].str.contains("11R"), "間取り"] = "1R"
    
    df_input.loc[df_input["築年数"].str.contains("520年5ヶ月"), "築年数"] = "52年5ヶ月"
    
    df_input.loc[df_input["築年数"].str.contains("1019年7ヶ月"), "築年数"] = "19年7ヶ月"
    
    df_input.loc[df_input["面積"]=="430.1m2", "面積"] = "43.01m2"
    #idx = df_input["築年数"].str.contains("1019年7ヶ月")
    #df_input["築年数"][idx] = "19年7ヶ月"    
    df_input.loc[df_input["面積"]==("1m2"), "面積"] = "10m2"
    df_input.loc[df_input["面積"]==("5.83m2"), "面積"] = "58.3m2"
    
    "三田線 三田(兵庫県)駅 徒歩14分"
    
    return df_input

all_df = modify_miss(all_df)

In [15]:
all_df["same_build"] = all_df[['building_floor', "所在地", 'int築年']].astype(str).sum(axis=1)

In [16]:
org_columns.values

array(['id', 'アクセス', 'キッチン', 'バス・トイレ', '周辺環境', '契約期間', '室内設備', '建物構造',
       '所在地', '所在階', '放送・通信', '方角', '築年数', '賃料', '間取り', '面積', '駐車場',
       '路線_0', '駅_0', '駅からの距離_0', '路線_1', '駅_1', '駅からの距離_1', '路線_2',
       '駅_2', '駅からの距離_2', '駅_0からの距離_mean', '駅_0からの距離_std', '駅_0からの距離_min',
       '駅_0からの距離_max', '駅_1からの距離_mean', '駅_1からの距離_std', '駅_1からの距離_min',
       '駅_1からの距離_max', '駅_2からの距離_mean', '駅_2からの距離_std', '駅_2からの距離_min',
       '駅_2からの距離_max', '路線_0_counts', '路線_1_counts', '路線_2_counts',
       '駅_0_counts', '駅_1_counts', '駅_2_counts', '路線_counts_max',
       '路線_counts_mean', '路線_counts_min', '路線_counts_std', '駅_counts_max',
       '駅_counts_mean', '駅_counts_min', '駅_counts_std', '距離_max',
       '距離_min', '距離_mean', '距離_std', '路線_0_counts/距離', '路線_1_counts/距離',
       '路線_2_counts/距離', '駅_0_counts/距離', '駅_1_counts/距離',
       '駅_2_counts/距離', '路線counts/距離_max', '路線counts/距離_mean',
       '路線counts/距離_min', '路線counts/距離_std', '駅counts/距離_max',
       '駅counts/距離_mean', '駅counts/距離_

In [17]:
%%time
targets = ['float面積', '築月数', 'contract_period', "building_floor", 'living_floor', 
           '設備和', 'int方角', "間取りtype", "int間取り", '駅からの距離_0', '駅からの距離_1',
           "駅からの距離_2", 'int建物構造']  #所在_区 での面積　あたりの値段

#targets = ['float面積', '築月数', 'contract_period', #'building_floor', 'living_floor', 'int方角',
#           #"間取りtype", "int間取り"
#]

columns = ["所在_区", "間取りtype", 'catuse_路線_0', 'catuse_駅_0', "int築年", #"地域_n丁目",
           "cat_int間取り", "定期借家", '一戸建て', "cat_building_height", #"same_build",
          ]
stats = ["mean", "std", "min", "max"]


for target in tqdm(targets):
    for column in columns:
        if target != column:
            for stat in stats:
                temp = all_df.groupby(column)[target].agg([stat]).rename({stat:f'{target}{stat}_of_{column}'},axis=1)
                #all_df = pd.merge(all_df,temp,on=column,how='left')
                all_df[f'{target}{stat}_of_{column}'] = all_df[column].map(temp.to_dict()[f'{target}{stat}_of_{column}'])
            if not (target == '築月数' and column == "int築年"):
                all_df[f'平均{target}_diff_{column}'] = all_df[f'{target}mean_of_{column}'] - all_df[target]
                all_df[f'平均{target}_percent_{column}'] = all_df[f'{target}mean_of_{column}'] / all_df[target]
                
all_df["面積/駅から距離"] = all_df["float面積"] / all_df['駅からの距離_0']    



CPU times: user 1min, sys: 1min, total: 2min
Wall time: 2min 1s


In [18]:
new_columns = [column for column in all_df.columns if column not in org_columns]

In [19]:
all_df_feature = all_df[new_columns]

In [20]:
all_df_feature = all_df_feature.drop(["same_build"], axis=1)
output_dir = '../code/feature_csv/'
all_df_feature.to_feather(os.path.join(output_dir, 'nonleak_group_feature2.feather'))

In [21]:
all_df_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62732 entries, 0 to 62731
Columns: 695 entries, float面積mean_of_所在_区 to 面積/駅から距離
dtypes: float64(607), int64(88)
memory usage: 332.6 MB


In [22]:
new_columns

['same_build',
 'float面積mean_of_所在_区',
 'float面積std_of_所在_区',
 'float面積min_of_所在_区',
 'float面積max_of_所在_区',
 '平均float面積_diff_所在_区',
 '平均float面積_percent_所在_区',
 'float面積mean_of_間取りtype',
 'float面積std_of_間取りtype',
 'float面積min_of_間取りtype',
 'float面積max_of_間取りtype',
 '平均float面積_diff_間取りtype',
 '平均float面積_percent_間取りtype',
 'float面積mean_of_catuse_路線_0',
 'float面積std_of_catuse_路線_0',
 'float面積min_of_catuse_路線_0',
 'float面積max_of_catuse_路線_0',
 '平均float面積_diff_catuse_路線_0',
 '平均float面積_percent_catuse_路線_0',
 'float面積mean_of_catuse_駅_0',
 'float面積std_of_catuse_駅_0',
 'float面積min_of_catuse_駅_0',
 'float面積max_of_catuse_駅_0',
 '平均float面積_diff_catuse_駅_0',
 '平均float面積_percent_catuse_駅_0',
 'float面積mean_of_int築年',
 'float面積std_of_int築年',
 'float面積min_of_int築年',
 'float面積max_of_int築年',
 '平均float面積_diff_int築年',
 '平均float面積_percent_int築年',
 'float面積mean_of_cat_int間取り',
 'float面積std_of_cat_int間取り',
 'float面積min_of_cat_int間取り',
 'float面積max_of_cat_int間取り',
 '平均float面積_diff_cat_int間取り',
 '平均float面積_perc